In [31]:
!pip3 install dash
!pip3 install dash-renderer
!pip3 install dash_core_components
!pip3 install dash_html_components
!pip3 install wget
!pip3 install jupyter-dash


print('\n installation complete')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: 

In [34]:
# Import required libraries
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

import wget



ContextualVersionConflict: (Jinja2 2.11.3 (/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages), Requirement.parse('Jinja2>=3.0'), {'flask'})

In [17]:
# Download the `spacex_launch_dash.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

print(min_payload, max_payload)
spacex_df.head()


0.0 9600.0


,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [26]:

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 
                                               'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites.
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                 dcc.Dropdown(id='site-dropdown',
                                              options=[
                                                       {'label': 'All Sites', 'value': 'ALL'},
                                                       {'label': 'Site 1', 'value': 'CCAFS LC-40'},
                                                       {'label': 'Site 1', 'value': 'CCAFS SLC-40'},
                                                       {'label': 'Site 1', 'value': 'KSC LC-39A'},
                                                       {'label': 'Site 1', 'value': 'VAFB SLC-4E'}
                                                      ],
                                                        value = 'ALL',
                                                        placeholder="Select a Launch Site here" ,
                                                        searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0,
                                                max = 10000,
                                                step = 1000,
                                                value = [min_payload, max_payload],
                                                marks = {i: i*1000 for i in range(11)} 
                                               ),
                                html,Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value')
             )
# Add computation to callback function and return graph
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(data_frame=filtered_df, values='class', names='pie chart names', title='Title: ')
        return fig
    else: 
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df.groupby(['Launch Site', 'class']).size().reset.index(name='class count')
        fig = px.pie(filtered_df, values = 'class count', name = 'class', title = 'Launch Success Count')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'), 
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')]
             )

def get_scatter(value1,value2):
    filtered_df2_1=spacex_df[(spacex_df['Payload Mass (kg)'] > value2[0]) & (spacex_df['Payload Mass (kg)'] < value2[1])]
    
    if value1=='All Sites':
        fig= px.scatter(filtered_df2_1,
                        x="Payload Mass (kg)",
                        y="class",
                        color="Booster Version Category",
                        title="Correlation between Payload and Success for All sites"
                       )
        return fig
    else:
        filtered_df2_2=filtered_df2_1[filtered_df2_1['Launch Site']==value1]
        fig= px.scatter(filtered_df2_2,
                        x="Payload Mass (kg)",
                        y="class",
                        color="Booster Version Category",
                        title=f"Correlation between Payload and Success for site {value1}"
                       )
        return fig
    

NameError: name 'dash' is not defined

In [2]:
# Run the app
if __name__ == '__main__':
    app.run_server()

NameError: name 'app' is not defined